<a href="https://colab.research.google.com/github/aj225patel/python-fundamentals/blob/main/advanced/decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Decorators**



*   *args is used to pass non-keyword, variable length argument list in the form of tuple
*   **kwargs is used to pass keyword, variable length argument list in the form of dict (key-value pairs)



https://www.python-engineer.com/courses/advancedpython/13-decorators/

In [8]:
def start_end_decorators(func):
  def wrapper(*args, **kwargs):
    print("start")
    func(*args, **kwargs)
    print("end")

  return wrapper

def print_name():
  print("Alex")

print_name()

Alex


In [9]:
print_name = start_end_decorators(print_name)
print_name()

start
Alex
end


In [10]:
@start_end_decorators
def is_age_valid(age: int):
  if age >= 18:
    print("Age is valid")
  else:
    print("Age is invalid")

is_age_valid(17)

start
Age is invalid
end


If we have a look at the name of our decorated function, and inspect it with the built-in help function, we notice that Python thinks our function is now the wrapped inner function of the decorator function.

In [11]:
print(help(is_age_valid))
print(is_age_valid.__name__)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

None
wrapper


In [13]:
import functools

def start_end_decorators_updated(func):
  @functools.wraps(func)              # preserved the information of used 'func'
  def wrapper(*args, **kwargs):
    print("start")
    func(*args, **kwargs)
    print("end")

  return wrapper

@start_end_decorators_updated
def add5(x):
  return x + 5

print(help(add5))
print(add5.__name__)

Help on function add5 in module __main__:

add5(x)

None
add5


## Decorators with arguments

> Note that functools.wraps is a decorator that takes an argument for itself. We can think of this as 2 inner functions, so an inner function within an inner function. To make this clearer, we look at another example: A repeat decorator that takes a number as input. Within this function, we have the actual decorator function that wraps our function and extends its behaviour within another inner function. In this case, it repeats the input function the given number of times.



### Execute Function N-number of times using decorator

In [15]:
import functools

def repeat(num_times: int):
  def decorator_repeat(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
      for _ in range(num_times):
        result = func(*args, **kwargs)
      return result
    return wrapper
  return decorator_repeat

n = 5

@repeat(num_times=n)
def greet(name: str):
  print(f"Hello, {name}")

greet('Alex')

Hello, Alex
Hello, Alex
Hello, Alex
Hello, Alex
Hello, Alex


## Nested Decorators

> We can apply several decorators to a function by stacking them on top of each other. The decorators are being executed in the order they are listed.



In [23]:
def start_end_decorator_2(func):

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper


def debug(func):
  @functools.wraps(func)
  def wrapper(*args, **kwargs):
    args_repr = [repr(a) for a in args]
    kwargs_repr = [f"{k}={v!r}" for k,v in kwargs.items()]
    signature = ", ".join(args_repr + kwargs_repr)
    print(f"calling {func.__name__}({signature})")
    result = func(*args, **kwargs)
    print(f"{func.__name__!r} returned {result!r}")
    return result
  return wrapper

@debug
@start_end_decorator_2
def say_hello(name: str) -> str:
  greeting = f"Hello, {name}"
  print(greeting)
  return greeting

say_hello(name='Alex')

calling say_hello(name='Alex')
Start
Hello, Alex
End
'say_hello' returned 'Hello, Alex'


'Hello, Alex'

## Class Decorators



We can also use a class as a decorator. Therefore, we have to implement the ______call__() method to make our object callable. Class decorators are typically used to maintain a state, e.g. here we keep track of the number of times our function is executed. The ______call__ method does essentially the same thing as the wrapper() method we have seen earlier. It adds some functionality, executes the function, and returns its result.
* Note that here we use functools.update_wrapper() instead of functools.wraps to preserve the information about our function.

In [26]:
import functools

class CountCalls:
    # the init needs to have the func as argument and stores it
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    # extend functionality, execute function, and return the result
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello(name: str):
    print(f"Hello! {name}")

say_hello('Andy')
say_hello('Gaurav')
say_hello('Vivan')

Call 1 of 'say_hello'
Hello! Andy
Call 2 of 'say_hello'
Hello! Gaurav
Call 3 of 'say_hello'
Hello! Vivan
